## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''

'''Resnet 50 架構'''
model = keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 28s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
_________________________

## 添加層數

In [2]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x = Dropout(p=0.1)(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 179


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    return X_train, X_test

## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
50000/50000 [==============================] - 446s 9ms/step - loss: 1.2388 - accuracy: 0.5871
Epoch 2/100
50000/50000 [==============================] - 443s 9ms/step - loss: 0.9947 - accuracy: 0.6595
Epoch 3/100
50000/50000 [==============================] - 443s 9ms/step - loss: 0.8904 - accuracy: 0.6942
Epoch 4/100
50000/50000 [==============================] - 443s 9ms/step - loss: 0.7889 - accuracy: 0.7265
Epoch 5/100
50000/50000 [==============================] - 443s 9ms/step - loss: 0.6625 - accuracy: 0.7669
Epoch 6/100
50000/50000 [==============================] - 443s 9ms/step - loss: 0.5994 - accuracy: 0.7936
Epoch 7/100
50000/50000 [==============================] - 444s 9ms/step - loss: 0.5271 - accuracy: 0.8169
Epoch 8/100
50000/50000 [==============================] - 444s 9ms/step - loss: 0.4785 - accuracy: 0.8320
Epoch 9/100
50000/50000 [==============================] - 444s 9ms/step - loss: 0.4717 - accuracy: 0.8335
Epoch 10/100
50000/50000 [===========

50000/50000 [==============================] - 430s 9ms/step - loss: 0.0708 - accuracy: 0.9786
Epoch 77/100
50000/50000 [==============================] - 430s 9ms/step - loss: 0.0743 - accuracy: 0.9776
Epoch 78/100
50000/50000 [==============================] - 431s 9ms/step - loss: 0.0663 - accuracy: 0.9796
Epoch 79/100
50000/50000 [==============================] - 433s 9ms/step - loss: 0.0657 - accuracy: 0.9797
Epoch 80/100
50000/50000 [==============================] - 428s 9ms/step - loss: 0.0632 - accuracy: 0.9812
Epoch 81/100
50000/50000 [==============================] - 428s 9ms/step - loss: 0.0605 - accuracy: 0.9815
Epoch 82/100
50000/50000 [==============================] - 431s 9ms/step - loss: 0.0628 - accuracy: 0.9807
Epoch 83/100
50000/50000 [==============================] - 442s 9ms/step - loss: 0.0621 - accuracy: 0.9807
Epoch 84/100
50000/50000 [==============================] - 437s 9ms/step - loss: 0.0623 - accuracy: 0.9810
Epoch 85/100
50000/50000 [===============